In [2]:
import sys
# pandas to manipulate SQL answer set
import pandas as pd
import time,os
# for Posgresql and other RDBMS
from sqlalchemy import create_engine,event,schema,Table,text
# from SQLAlchemy import  
from sqlalchemy_utils import database_exists,create_database
from sqlalchemy.orm import sessionmaker
import psycopg2
import matplotlib.pyplot as plt

In [72]:

# Connect the postgres database from your local machine using psycopg2

connection.autocommit = True
# Create a cursor to perform database operations
cursor = connection.cursor()

# Print PostgreSQL details
print("PostgreSQL server information")
print(connection.get_dsn_parameters(), "\n")
# Executing a SQL query
cursor.execute("SELECT version();")
# Fetch result
record = cursor.fetchone()
print("You are connected to - ", record, "\n")
cursor.execute(
    '''select table_name from information_schema.tables where table_schema = 'public'; '''
)
for table in cursor.fetchall():
    print(table)
cursor.close()

PostgreSQL server information
{'user': 'grp15_2023', 'dbname': 'grp15db_2023', 'host': 'dbcourse.cs.aalto.fi', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 12.15 (Ubuntu 12.15-0ubuntu0.20.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0, 64-bit',) 

('patient',)
('vaccine',)
('manufacturer',)
('batch',)
('healthcarefacility',)
('attends',)
('staff',)
('shift',)
('transportationlog',)
('vaccination',)
('diagnosis',)
('symptoms',)
('patientsymtpoms',)


In [73]:
DIALECT = 'postgresql+psycopg2://'
db_uri = "%s:%s@%s/%s" % (user, password, host, database)

engine = create_engine(DIALECT + db_uri)
psql_conn = engine.connect()

if not psql_conn:
    print("DB connection is not OK!") 
else:
    print("DB connection is OK.")

DB connection is OK.


In [75]:
query = text('''SELECT DISTINCT Staff.ssNo, Staff.name, Staff.phone, Staff.role, Staff.vaccinationStatus, healthcarefacility.name 
FROM
vaccination
JOIN healthcarefacility ON healthcarefacility.name = Vaccination.healthcareFacilityName 
JOIN Staff ON healthcarefacility.name = Staff.healthcareFacilityName 
JOIN Shift ON Shift.ssNo = Staff.ssNo
WHERE vaccination.date = '2021-05-10' AND 1 = shift.workday;''')

result = pd.read_sql_query(query, psql_conn)
print(result)

            ssno               name         phone    role  vaccinationstatus  \
0  19630812-6581   Jazlyn Schneider  040-868-2528   nurse                  1   
1  19720223-1761   Alfreda Champlin  041-631-1851   nurse                  1   
2  19740919-7140         Deon Hoppe  040-399-1121   nurse                  0   
3  19771003-5988        Samir Hills  040-093-0059   nurse                  1   
4  19820218-5928      Elena Bartell  041-938-9451   nurse                  1   
5  19880817-8027  Haylie Wintheiser  050-448-8894   nurse                  1   
6  19920802-4854        Kaden Tromp  044-624-1591   nurse                  1   
7  19940615-4448      Jordy Hilpert  044-506-1982  doctor                  1   

                         name  
0  Sanomala Vaccination Point  
1     Myyrmäki Energia Areena  
2       Tapiola Health Center  
3  Sanomala Vaccination Point  
4     Myyrmäki Energia Areena  
5     Myyrmäki Energia Areena  
6       Tapiola Health Center  
7       Tapiola Health 

            ssNo               name         phone    role  vaccinationStatus      name 
0  19630812-6581   Jazlyn Schneider  040-868-2528   nurse                1.0      Sanomala Vaccination Point 
1  19720223-1761   Alfreda Champlin  041-631-1851   nurse                1.0      Myyrmäki Energia Areena
2  19740919-7140         Deon Hoppe  040-399-1121   nurse                0.0      Tapiola Health Center
3  19771003-5988        Samir Hills  040-093-0059   nurse                1.0      Sanomala Vaccination Point
4  19820218-5928      Elena Bartell  041-938-9451   nurse                1.0      Myyrmäki Energia Areena
5  19880817-8027  Haylie Wintheiser  050-448-8894   nurse                1.0      Myyrmäki Energia Areena 
6  19920802-4854        Kaden Tromp  044-624-1591   nurse                1.0      Tapiola Health Center 
7  19940615-4448      Jordy Hilpert  044-506-1982  doctor                1.0      Tapiola Health Center 

In [135]:
query1 = text('''SELECT DISTINCT shift.ssNo, name, role, healthcarefacilityname FROM Staff 
                JOIN Shift ON Staff.ssNo = Shift.ssNo 
                WHERE Shift.workday = 3 AND Staff.role = 'doctor' 
                AND (Staff.healthcareFacilityName = 'Messukeskus' OR Staff.healthcareFacilityName = 'Malmi');''')

result1 = pd.read_sql_query(query1, psql_conn)
print(result1)


            ssno              name    role healthcarefacilityname
0  19740731-5488   Rosalia Simonis  doctor            Messukeskus
1  19750726-4531      Shaylee Kris  doctor            Messukeskus
2  19751212-3265     Hilbert Purdy  doctor            Messukeskus
3  19760102-8374  Elnora Greenholt  doctor            Messukeskus


            ssNo              name
0  19740731-5488   Rosalia Simonis
1  19750726-4531      Shaylee Kris
2  19751212-3265     Hilbert Purdy
3  19760102-8374  Elnora Greenholt

In [47]:
query1 = text('''SELECT B.id AS batchID, B.healthcarefacilityname AS currentLocation, TL.tolocation AS lastLocation,
  CASE WHEN B.healthcarefacilityname = TL.tolocation THEN '' ELSE HF.phone END AS phone
FROM Batch AS B
JOIN TransportationLog AS TL ON B.id = TL.batchid
JOIN HealthcareFacility AS HF ON B.healthcarefacilityname = HF.name
WHERE TL.arrivaldate = (
  SELECT MAX(arrivaldate)
  FROM TransportationLog
  WHERE batchid = B.id
)
AND B.healthcarefacilityname != TL.tolocation

UNION

SELECT B.id AS batchID, B.healthcarefacilityname AS currentLocation, TL.tolocation AS lastLocation, ''
FROM Batch AS B
JOIN TransportationLog AS TL ON B.id = TL.batchid
JOIN HealthcareFacility AS HF ON B.healthcarefacilityname = HF.name
WHERE TL.arrivaldate = (
  SELECT MAX(arrivaldate)
  FROM TransportationLog
  WHERE batchid = B.id
)
AND B.healthcarefacilityname = TL.tolocation;''')

result1 = pd.read_sql_query(query1, psql_conn)
print(result1)

   batchid              currentlocation                 lastlocation  \
0     B01    Sanomala Vaccination Point   Sanomala Vaccination Point   
1     B02                   Messukeskus   Sanomala Vaccination Point   
2     B03       Myyrmäki Energia Areena      Myyrmäki Energia Areena   
3     B04                         Malmi                        Malmi   
4     B06   Iso Omena Vaccination Point      Myyrmäki Energia Areena   
5     B07       Myyrmäki Energia Areena      Myyrmäki Energia Areena   
6     B08         Tapiola Health Center        Tapiola Health Center   
7     B12    Sanomala Vaccination Point   Sanomala Vaccination Point   
8     B13   Iso Omena Vaccination Point  Iso Omena Vaccination Point   
9     B15                         Malmi                        Malmi   
10    B16         Tapiola Health Center        Tapiola Health Center   
11    B17       Myyrmäki Energia Areena      Myyrmäki Energia Areena   
12    B18         Tapiola Health Center        Tapiola Health Ce

   batchid              currentlocation                 lastlocation    phone 
0     B01    Sanomala Vaccination Point   Sanomala Vaccination Point
1     B02                   Messukeskus   Sanomala Vaccination Point    093-101-0024  
2     B03       Myyrmäki Energia Areena      Myyrmäki Energia Areena
3     B04                         Malmi                        Malmi    
4     B06   Iso Omena Vaccination Point      Myyrmäki Energia Areena    098-163-4500
5     B07       Myyrmäki Energia Areena      Myyrmäki Energia Areena
6     B08         Tapiola Health Center        Tapiola Health Center
7     B12    Sanomala Vaccination Point   Sanomala Vaccination Point
8     B13   Iso Omena Vaccination Point  Iso Omena Vaccination Point
9     B15                         Malmi                        Malmi
10    B16         Tapiola Health Center        Tapiola Health Center
11    B17       Myyrmäki Energia Areena      Myyrmäki Energia Areena
12    B18         Tapiola Health Center        Tapiola Health Center
13    B21   Iso Omena Vaccination Point  Iso Omena Vaccination Point
14    B22       Myyrmäki Energia Areena      Myyrmäki Energia Areena
15    B23    Sanomala Vaccination Point   Sanomala Vaccination Point
16    B24                         Malmi                        Malmi
17    B25                         Malmi                        Malmi
18    B27       Myyrmäki Energia Areena      Myyrmäki Energia Areena
19    B28   Iso Omena Vaccination Point  Iso Omena Vaccination Point
20    B29       Myyrmäki Energia Areena   Sanomala Vaccination Point    093-104-5930 
21    B30   Iso Omena Vaccination Point  Iso Omena Vaccination Point

In [137]:
query1 = text('''
SELECT *
FROM Patient p
JOIN Diagnosis d ON p.ssNo = d.ssNo
JOIN Symptoms s ON d.symptom = s.name
WHERE s.criticality = 1 AND d.date > '2021-05-10';''')
result1 = pd.read_sql_query(query1, psql_conn)
print(result1)

Empty DataFrame
Columns: [ssno, name, birthday, gender, vaccinationstatus, ssno, symptom, date, name, criticality]
Index: []


In [48]:
psql_conn.execute(text('''
CREATE VIEW vaccination_status_view AS
SELECT P.ssno, P.name, P.birthday, P.gender,
       CASE
           WHEN COUNT(A.*) >= vc.dosage THEN 1
           ELSE 0
       END AS vaccinationStatus
FROM Patient P
LEFT JOIN Attends A ON P.ssNo = A.ssNo
LEFT JOIN Vaccination V ON V.date = A.date AND V.healthcarefacilityname = A.healthcarefacilityname
LEFT JOIN Batch B ON B.id = V.batchid
LEFT JOIN Manufacturer M ON M.id = B.manufacturerid
LEFT JOIN Vaccine VC ON VC.id = M.vaccineid
GROUP BY P.ssNo, VC.dosage;'''))
query = text('SELECT * FROM vaccination_status_view;')
result = pd.read_sql_query(query, psql_conn)

pd.set_option('display.max_rows', None)  # Set the display option to show all rows
print(result)

            ssno                        name    birthday gender  \
0    850310-787I               Cristal Borer  1985-03-10      M   
1    960807-613R                Omer Denesik  1996-08-07      M   
2    871128-519R                Flossie Torp  1987-11-28      F   
3    901010-3010             Emerald Johnson  1990-10-10      F   
4    140307-203V              Rhea Hettinger  2014-03-07      M   
5    050922-330C            Mrs. Bulah Grant  2005-09-22      F   
6    840805-1135               Lonzo Collier  1984-08-05      M   
7    060927-1438            Zetta Runolfsson  2006-09-27      F   
8    940214-8179           Glenna Collier II  1994-02-14      F   
9    141005-8066         Dr. Rhea Schaefer I  2014-10-05      M   
10   160902-592P             Britney Gutmann  2016-09-02      F   
11   050218-4796   Ms. Hassie Runolfsson PhD  2005-02-18      F   
12   090518-869W               Spencer Kunde  2009-05-18      M   
13   770504-8289             Andreanne Stamm  1977-05-04      

            ssno                        name    birthday gender  \						
0    850310-787I               Cristal Borer  1985-03-10      M   	                  0  	
1    960807-613R                Omer Denesik  1996-08-07      M   	                  0  	
2    871128-519R                Flossie Torp  1987-11-28      F   	                  1  	
3    901010-3010             Emerald Johnson  1990-10-10      F   	                  0  	
4    140307-203V              Rhea Hettinger  2014-03-07      M   	                  0  	
5    050922-330C            Mrs. Bulah Grant  2005-09-22      F   	                  0  	
6    840805-1135               Lonzo Collier  1984-08-05      M   	                  1  	
7    060927-1438            Zetta Runolfsson  2006-09-27      F   	                  0  	
8    940214-8179           Glenna Collier II  1994-02-14      F   	                  0  	
9    141005-8066         Dr. Rhea Schaefer I  2014-10-05      M   	                  0  	
10   160902-592P             Britney Gutmann  2016-09-02      F   	                  0  	
11   050218-4796   Ms. Hassie Runolfsson PhD  2005-02-18      F   	                  0  	
12   090518-869W               Spencer Kunde  2009-05-18      M   	                  0  	
13   770504-8289             Andreanne Stamm  1977-05-04      F   	                  0  	
14   951217-488K                 Casey Brown  1995-12-17      M   	                  0  	
15   810616-9029         Dr. Hester Veum PhD  1981-06-16      M   	                  0  	
16   090602-7898             Hillard Boehm V  2009-06-02      M   	                  0  	
17   820428-701R              Dallas Collins  1982-04-28      M   	                  0  	
18   990622-5231                    Sid Hahn  1999-06-22      M   	                  0  	
19   090202-1778             Alvera Medhurst  2009-02-02      F   	                  0  	
20   801022-152O              Dane Barrows I  1980-10-22      F   	                  0  	
21   060925-8919      Prof. Garry Renner Sr.  2006-09-25      M   	                  0  	
22   120203-4446           Mr. Oren Block IV  2012-02-03      M   	                  0  	
23   000425-224A           Christian McGlynn  2000-04-25      M   	                  0  	
24   701127-5340         Mrs. Lorena Kreiger  1970-11-27      F   	                  0  	
25   021001-957O                 John Larkin  2002-10-01      M   	                  0  	
26   060325-323X              Darlene Brakus  2006-03-25      F   	                  0  	
27   720214-2797               Abbey Schuppe  1972-02-14      F   	                  0  	
28   761223-707W            Jacinto Prohaska  1976-12-23      M   	                  0  	
29   141231-4486              Kristina Pagac  2014-12-31      F   	                  0  	
30   820126-623A     Miss Rosalyn Reichel MD  1982-01-26      F   	                  0  	
31   801127-1830               Ms. Ivy Nolan  1980-11-27      F   	                  0  	
32   731122-126T             Johanna McClure  1973-11-22      F   	                  0  	
33   041208-5838     Dr. Margarette Mertz IV  2004-12-08      F   	                  0  	
34   780214-1893       Prof. Erling Morar MD  1978-02-14      F   	                  0  	
35   151129-922D                 Ned Ziemann  2015-11-29      F   					  0  	
36   950303-191X       Dr. Simeon Keeling II  1995-03-03      M   					  0  	
37   860811-3456                  Syble Howe  1986-08-11      F   					  0  	
38   040113-8197              Santiago Hintz  2004-01-13      M   					  0  	
39   190709-8467            Zander Powlowski  2019-07-09      M   					  0  	
40   801108-303B                Reva Waelchi  1980-11-08      F   					  0  	
41   851228-732X              Shanna Osinski  1985-12-28      F   					  0  	
42   741104-114G                Marvin Fahey  1974-11-04      M   					  0  	
43   000127-4899               Ms. Opal Lang  2000-01-27      F   					  0  	
44   891214-962C           Clifton Boyle DDS  1989-12-14      M   					  0  	
45   820906-808J       Prof. Paula Schaden I  1982-09-06      F   					  0  	
46   721030-5216               Noah Leuschke  1972-10-30      M   					  0  	
47   070101-326Q              Orpha Bogisich  2007-01-01      F   					  0  	
48   010327-525G         Dr. Mireille Hansen  2001-03-27      M   					  1  	
49   710327-313B            Prof. Ewell Conn  1971-03-27      M   					  0  	
50   710325-877I              Nicolette Moen  1971-03-25      F   					  0  	
51   190615-6325        Ms. Nelda Brekke PhD  2019-06-15      F   					  0  	
52   090416-443L               Aliyah Harber  2009-04-16      M   					  1  	
53   790608-9686            Gisselle Hilpert  1979-06-08      F   					  0  	
54   190604-440B        Prof. Kevon Cummings  2019-06-04      M   					  0  	
55   821213-6162                Tabitha Howe  1982-12-13      M   					  0  	
56   850212-376B       Mr. Ellis Jaskolski I  1985-02-12      M   					  0  	
57   010201-5814             Harrison Heaney  2001-02-01      M   					  0  	
58   720621-378H            Ms. Alisha Ortiz  1972-06-21      F   					  0  	
59   830908-9826                    Ana Ward  1983-09-08      F   					  0  	
60   120318-5791              Mathilde Smith  2012-03-18      F   					  0  	
61   971214-2818        Prof. Brice Metz PhD  1997-12-14      M   					  0  	
62   871118-242U        Dr. Victor Armstrong  1987-11-18      M   					  0  	
63   130701-235N             Arnold Medhurst  2013-07-01      M   					  0  	
64   710724-5480         Dashawn Schamberger  1971-07-24      M   					  0  	
65   730927-7319        Mr. Gunner O'Hara MD  1973-09-27      M   					  0  	
66   011103-6045     Mrs. Kailyn Collier DVM  2001-11-03      F   					  0  	
67   780807-657T        Prof. Jorge Ward Jr.  1978-08-07      M   					  0  	
68   960629-4156               Rossie Spinka  1996-06-29      F   					  0  	
69   881006-6543            Trycia Jaskolski  1988-10-06      F   					  0  	
70   171125-8471            Lucie Walter PhD  2017-11-25      F   					  0  	
71   751211-287B              Taylor Krajcik  1975-12-11      F   					  1  	
72   150419-7928               Ona Crona III  2015-04-19      M   					  0  	
73   180604-6419      Dr. Freddie Cartwright  2018-06-04      M   					  0  	
74   130205-474D              Elenora Sawayn  2013-02-05      F   					  0  	
75   060729-292C                Marilou Ryan  2006-07-29      F   					  0  	
76   000325-6271              Mariam Ritchie  2000-03-25      F   					  0  	
77   130725-3724              Shemar Bradtke  2013-07-25      F   					  0  	
78   710422-6080           Mr. Ike Wyman III  1971-04-22      M   					  0  	
79   980626-9033              Leonie Dibbert  1998-06-26      M   					  0  	
80   701117-199K                 Leland Moen  1970-11-17      M   					  0  	
81   100825-914H               Astrid Willms  2010-08-25      F   					  0  	
82   760108-8906            Miss Helen Boyer  1976-01-08      F   					  0  	
83   720414-668E                  Paige Conn  1972-04-14      F   					  0  	
84   841229-112N            Rodolfo O'Reilly  1984-12-29      M   					  1  	
85   840301-7572               Maxine Russel  1984-03-01      M   					  0  	
86   210318-737O                 Corine Hane  2021-03-18      F   					  1  	
87   160930-586P               Aiden Volkman  2016-09-30      F   					  1  	
88   090502-3556        Mr. Dereck Brekke IV  2009-05-02      M   					  0  	
89   021201-3734        Prof. Raymond Beahan  2002-12-01      M   					  0  	
90   080305-985A                  Ricky Kuhn  2008-03-05      M   					  0  	
91   011112-5614              Magnus Gerhold  2001-11-12      M   					  0  	
92   161215-9509           Roslyn Rempel DDS  2016-12-15      F   					  0  	
93   970708-7885                  Ilene Kris  1997-07-08      F   					  0  	
94   150601-1657     Mrs. Ophelia Corwin Sr.  2015-06-01      F   					  0  	
95   760823-949J           Helene Schiller I  1976-08-23      F   					  0  	
96   990903-6514               Eldred Blanda  1999-09-03      F   					  0  	
97   110420-6983             Nellie Nitzsche  2011-04-20      F   					  0  	
98   710113-8500        Dr. Geovanni Krajcik  1971-01-13      M   					  0  	
99   730126-956K              Kayden Farrell  1973-01-26      M   					  0  	
100  090226-5673            Dr. Lamont Ferry  2009-02-26      M   					  1  	
101  060421-302M             Faustino Barton  2006-04-21      M   					  1  	
102  041122-6308          Prof. Demarco Hahn  2004-11-22      M   					  0  	
103  890607-6113  Prof. Raphael Prosacco DVM  1989-06-07      M   					  0  	
104  850315-155F        Andreanne Jakubowski  1985-03-15      M   					  0  	
105  970801-218B              Jonathan Wyman  1997-08-01      M   					  0  	
106  830820-576C                Loyal Hoeger  1983-08-20      M   					  0  	
107  890104-753F       Lukas Runolfsdottir V  1989-01-04      M   					  1  	
108  021130-649D       Dr. Jayson Glover DVM  2002-11-30      M   					  0  	
109  181012-5367           Bulah Heidenreich  2018-10-12      M   					  0  	
110  150416-255C             Jacklyn Padberg  2015-04-16      F   					  0  	
111  881210-971J             Brain Greenholt  1988-12-10      M   					  0  	
112  071001-8429          Dr. Macie Ondricka  2007-10-01      F   					  0  	
113  930508-413K          Mr. Reid Little II  1993-05-08      M   					  0  	
114  040208-7391          Prof. Sadye Walter  2004-02-08      F   					  0  	
115  841026-9331                 Brad Lehner  1984-10-26      M   					  0  	
116  881119-6103      Destiny Konopelski PhD  1988-11-19      F   					  0  	
117  120407-897G                 Torey Mills  2012-04-07      M   					  0  	
118  110614-978B           Ms. Hanna Corkery  2011-06-14      F   					  0  	
119  090707-295R          Alysson Jakubowski  2009-07-07      F   					  0  	
120  780613-7560                   Silas Fay  1978-06-13      F   					  0  	
121  880706-240U  Prof. Willard Marquardt II  1988-07-06      M   					  0  	
122  041030-267L               Isabell Nader  2004-10-30      F   					  0  	
123  041113-8113   Prof. Yessenia Dooley Jr.  2004-11-13      F   					  0  	
124  790503-394M               Kathlyn Moore  1979-05-03      F   					  0  	
125  930804-7021          Prof. Harrison Toy  1993-08-04      M   					  0  	
126  720801-8748             Braeden Hackett  1972-08-01      M   					  0  	
127  701202-2506           Godfrey Haley Sr.  1970-12-02      M   					  0  	
128  790121-7800                Otilia Stamm  1979-01-21      M   					  0  	
129  070117-2839              Estrella Johns  2007-01-17      F   					  0  	
130  060406-686D                  Dovie West  2006-04-06      F   					  0  	
131  070218-9109              Angelina White  2007-02-18      F   					  0  	
132  760708-708H               Maia Towne II  1976-07-08      M   					  0  	
133  031127-2165            Mathew Buckridge  2003-11-27      M   					  0  	
134  101229-7602             Amelie Cummings  2010-12-29      F   					  0  	
135  990614-395X      Josefa Greenfelder DVM  1999-06-14      M   					  0  	
136  880810-358W                Braxton Hane  1988-08-10      M   					  1  	
137  730218-253D                 Dereck Beer  1973-02-18      M   					  0  	
138  140620-7388               Tina Kshlerin  2014-06-20      F   					  0  	
139  930804-509I                    Fay Ryan  1993-08-04      F   					  0  	
140  860804-829D             Miss Hope Jones  1986-08-04      F   					  0  	
141  811005-5518      Miss Charity Powlowski  1981-10-05      F   					  0  	
142  930106-189U                Julius Marks  1993-01-06      M   					  0  	
143  000506-642P            Cassandra Mayert  2000-05-06      F   					  0  	
144  950803-1903                  Lyric Funk  1995-08-03      M   					  0  	
145  011119-9865               Ahmad Kovacek  2001-11-19      M   					  0  	
146  130704-908X                  Baby Kunde  2013-07-04      M   					  0  	
147  110929-252V             Lilly Farrell V  2011-09-29      F   					  0  	
148  080514-3385          Dakota Greenfelder  2008-05-14      F   					  0  	
149  741222-8947               Devon Nicolas  1974-12-22      M   					  0  	
																																														
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						
						

In [49]:
query1 = text('''SELECT hf.name AS facility_name, 
       SUM(b.numberOfVaccines) AS total_vaccines,
       COUNT(DISTINCT v.id) AS different_types
FROM HealthcareFacility hf
LEFT JOIN Batch b ON hf.name = b.healthcareFacilityName
LEFT JOIN Manufacturer m ON b.manufacturerID = m.id
LEFT JOIN Vaccine v ON m.vaccineID = v.id
GROUP BY hf.name;''')
result1 = pd.read_sql_query(query1, psql_conn)
print(result1)

                 facility_name  total_vaccines  different_types
0  Iso Omena Vaccination Point              65                3
1                        Malmi              65                3
2                  Messukeskus             120                3
3      Myyrmäki Energia Areena              85                3
4   Sanomala Vaccination Point              40                2
5        Tapiola Health Center              55                2


In [50]:
query1 = text('''SELECT Vaccine.id, Symptoms.name, COUNT(*) as frequency,
       COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Vaccine.id) as relative_frequency
FROM Diagnosis
JOIN (Attends NATURAL JOIN Vaccination) ON Diagnosis.ssNo = Attends.ssNo
JOIN Batch ON Batch.ID = Vaccination.batchID
JOIN Manufacturer ON Manufacturer.id = Batch.manufacturerID
JOIN Vaccine ON Vaccine.id = Manufacturer.vaccineID
JOIN Symptoms ON Symptoms.name = Diagnosis.symptom
WHERE Diagnosis.date >= Attends.date
GROUP BY Vaccine.id, Symptoms.name;''')
result1 = pd.read_sql_query(query1, psql_conn)
print(result1)

      id                         name  frequency  relative_frequency
0   V01            blurring of vision          1            2.325581
1   V01                      diarrhea          1            2.325581
2   V01                       fatigue          1            2.325581
3   V01           feelings of illness          1            2.325581
4   V01                         fever          3            6.976744
5   V01                      headache          7           16.279070
6   V01                    high fever          2            4.651163
7   V01   inflammation near injection          1            2.325581
8   V01      itchiness near injection          4            9.302326
9   V01                    joint pain          6           13.953488
10  V01                   muscle ache          8           18.604651
11  V01                        nausea          4            9.302326
12  V01                      vomiting          1            2.325581
13  V01         warmth near inject

In [19]:
psql_conn.close()
connection.close()